In [ ]:
#Hide warnings
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf

In [ ]:
from tensorflow import keras
accumulatorc = keras.models.load_model('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/weights/accumulator.h5')
coolerc = keras.models.load_model('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/weights/cooler.h5')
stablec= keras.models.load_model('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/weights/stable.h5')
valvec = keras.models.load_model('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/weights/valve.h5')
leakagec = keras.models.load_model('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/weights/leakage.h5')
rulc = keras.models.load_model('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/weights/rul.h5')

In [ ]:
# load the dataset 
import pandas as pd
import numpy as np

dt = pd.read_excel('/content/drive/MyDrive/Pump Predictive Maintenance/Condition Hydraulic Pump/condition Hydraulic.xlsx')

In [ ]:
df = pd.DataFrame(dt, columns=['PS1','PS3', 'PS4', 'PS5', 'FS1', 'FS2', 'TS1', 'P1', 'VS1', 'CE1', 'CP1', 'SE1', 'leakage', 'valve', 'cooler', 'accumulator', 'rul', 'label1'])
df.sample(10)

,PS1,PS3,PS4,PS5,FS1,FS2,TS1,P1,VS1,CE1,CP1,SE1,leakage,valve,cooler,accumulator,rul,label1
2114,160.671200,1.949312,10.116157,9.892020,6.495867,10.159338,35.773033,2549.226767,0.544267,46.978833,2.161833,56.931617,1,100,100,90,0.166667,1
1041,158.755557,1.787245,0.000000,9.060099,6.667395,9.663312,45.710150,2460.608333,0.615367,27.782233,1.752250,59.567200,0,80,20,130,0.000000,1
971,157.900827,1.715422,0.000000,9.023155,6.361792,9.626272,46.008683,2466.177267,0.631917,27.274100,1.713100,56.723067,2,90,20,130,0.000000,1
543,172.784225,1.084655,0.000000,8.377100,3.333893,8.882052,57.584167,2607.361433,0.727367,19.025967,1.499833,31.148750,1,90,3,100,0.000000,1
1511,160.998580,2.013957,0.710880,9.967198,6.685910,10.157025,35.435017,2545.970800,0.551917,46.430750,2.147617,58.818200,0,100,100,90,27.333333,0
1152,158.453738,1.759557,0.000000,9.137215,6.478800,9.688445,44.565867,2467.361067,0.610367,27.745767,1.753617,57.829967,1,100,20,115,0.000000,1
1984,160.730637,1.935376,10.051590,9.824836,6.463323,10.140378,36.350300,2549.314733,0.538683,46.433317,2.124350,56.053550,1,73,100,100,0.000000,1
599,156.148142,1.722848,0.000000,8.418393,6.557792,8.949050,56.901350,2370.323300,0.717533,18.739900,1.463583,59.834150,0,100,3,90,1.833333,0
670,172.878865,1.095611,0.000513,8.402254,3.295628,8.902225,57.155267,2608.158167,0.726933,18.844583,1.495300,30.436850,1,80,3,90,0.000000,1
1146,158.527948,1.763424,0.000000,9.145822,6.474310,9.695677,44.629483,2477.409467,0.623267,27.655250,1.752650,57.584983,1,90,20,115,0.000000,1


In [ ]:
print(df['leakage'].unique())
print(df['cooler'].unique())
print(df['accumulator'].unique())
print(df['valve'].unique())
print(df['label1'].unique())

[0 2 1]
[  3  20 100]
[130 115 100  90]
[100  73  80  90]
[0 1]


In [ ]:
X = df.iloc[:, :12]
y = df.iloc[:, 15]

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
df.sample(10)

,PS1,PS3,PS4,PS5,FS1,FS2,TS1,P1,VS1,CE1,CP1,SE1,leakage,valve,cooler,accumulator,rul,label1
1879,160.565385,1.948675,10.098277,9.866102,6.523953,10.176417,36.169700,2553.126133,0.546517,46.979367,2.161217,57.167967,1,100,100,115,0.000000,1
1806,160.894388,1.999104,10.071829,9.839482,6.702407,10.170137,36.321450,2536.757733,0.544217,47.101517,2.155650,59.225083,0,100,100,115,0.333333,1
838,158.277698,1.799744,0.000000,8.998756,6.636357,9.613428,46.788550,2448.512167,0.600250,26.324217,1.710000,59.565217,0,100,20,90,17.500000,0
160,156.951137,1.738583,0.000000,8.564248,6.613407,9.214733,53.898833,2400.569667,0.656833,19.713417,1.545383,60.585850,0,100,3,130,8.500000,0
921,158.406957,1.777062,0.000000,8.998314,6.636820,9.615177,46.422317,2443.548133,0.625067,26.394733,1.716900,59.751300,0,100,20,90,3.666667,0
855,158.308372,1.792338,0.000000,8.993964,6.639383,9.615605,46.649050,2446.103567,0.593900,26.385250,1.713050,59.729733,0,100,20,90,14.666667,0
1639,161.061293,2.002360,2.583576,9.965819,6.684578,10.211078,35.421117,2547.796533,0.547200,46.987233,2.170733,58.742900,0,100,100,90,6.000000,0
1878,160.610177,1.938668,10.093895,9.863730,6.520082,10.177712,36.187783,2553.700133,0.539883,47.163433,2.165833,57.008067,1,90,100,115,0.000000,1
2037,161.097873,2.000081,10.076048,9.846903,6.686412,10.144083,36.221583,2539.788767,0.544283,47.115450,2.171400,58.797633,0,80,100,100,0.000000,1
757,157.830742,1.762823,0.000000,8.906859,6.635380,9.523257,48.255050,2437.580967,0.600233,26.127533,1.744600,59.680617,0,100,20,90,31.000000,0


In [ ]:
a=np.array([[160.610177,	1.938668,	10.093895,	9.863730,	6.520082,	10.177712,	36.187783,	2553.700133,	0.539883,	47.163433,	2.165833,	57.008067]])
ac=sc.transform(a)

In [ ]:
failure = np.argmax(stablec.predict(ac), axis=-1)
stab=['stable', 'failure']
print("The Hydraulic pump is {}".format(stab[failure[0]]))
cooler = np.argmax(coolerc.predict(ac), axis=-1)
cool=['3(close to failure)', '20(reduced efficiency)', '100(full efficiency)']
print("Condition of cooler is {}".format(cool[cooler[0]]))
leakage = np.argmax(leakagec.predict(ac), axis=-1)
leak =['0 (No leak)', '1 (weak leak)','2 (severe leak)']
print("Condition of Internal Pump Leakage is {}".format(leak[leakage[0]]))
valve = np.argmax(valvec.predict(ac), axis=-1)
val =['73 (close to total failure)', '80 (Severe lag)', '90 (small lag)', '100 (optimal switching behaviour)']
print("Condition of Valve is {}".format(val[valve[0]]))
accumulator = np.argmax(accumulatorc.predict(ac), axis=-1)
accu= ['90 (close to total failure)', '100 (severely reduced pressure)', '115 (slightly reduced pressure)', '130 (optimal pressure)']
print("Condition of Hydraulic accumulator is {}".format(accu[accumulator[0]]))
rul=rulc.predict([ac])
print("Remaining useful life before failure is {} minutes".format(max(0, rul[0])))

The Hydraulic pump is failure
Condition of cooler is 100(full efficiency)
Condition of Internal Pump Leakage is 1 (weak leak)
Condition of Valve is 100 (optimal switching behaviour)
Condition of Hydraulic accumulator is 130 (optimal pressure)
Remaining useful life before failure is [0.01968958] minutes
